In [2]:
import requests
from bs4 import BeautifulSoup

url = 'https://comic.naver.com/webtoon/weekday.nhn'

response = requests.get(url)
response.status_code
# print(response.text)
soup = BeautifulSoup(response.text)
# print(soup)

In [3]:
dragon = 568986
webtoon_detail(dragon)

{'title': '용이산다',
 'description': '드래곤이든 인간이든 그저 평범한 판타지 일상물.\n새로운 용(?)물 등장!',
 'author': '초',
 'genre': ['스토리', '일상', '판타지'],
 'age': '전체연령가'}

In [12]:
import re
import requests
from bs4 import BeautifulSoup


menu = '''# 웹툰 크롤러 #
1. 정보
2. 에피소드 목록
0. 나가기'''
# print(menu)


info_msg = '''# 정보를 가져올 웹툰 선택'''
# print(info_msg)
episode_list_msg = '''# 에피소드 목록 ({title})'''
# print(episode_list_msg)


def webtoon_info(title):
    soup = BeautifulSoup(requests.get('https://comic.naver.com/webtoon/weekday.nhn').text)
#     print(soup)
    a_list = soup.select('a.title[title*="{}"]'.format(title))
#     print(a_list)

    results = []
    
    for a in a_list:
        href = a['href']
        m = re.search(r'titleId=(\d+)', href)
        title_id = m.group(1)
        thumbnail = a.parent.select_one('img')['src']
        title = a.get_text(strip=True)
#         print(title)
#         print()

        cur_info = {
            'title': title,
            'title_id': title_id,
            'link': href,
            'thumbnail': thumbnail
        }
        results.append(cur_info)
        
    return results


def webtoon_detail(title_id):
    url = f'https://comic.naver.com/webtoon/list.nhn?titleId={title_id}'
    response = requests.get(url)
    html = response.text
#     print(html)
    soup = BeautifulSoup(html)
    
    div_comicinfo = soup.select_one('div.comicinfo')
    div_detail = div_comicinfo.select_one('div.detail')
    
    title = div_detail.select_one('h2').contents[0].strip()
    author = div_detail.select_one('span.wrt_nm').get_text(strip=True)
    description = div_detail.select_one('p').get_text('\n', strip=True)
    
    div_detail_info = div_detail.select_one('p.detail_info')
    genre = div_detail_info.select_one('span.genre').get_text(strip=True)
    age = div_detail_info.select_one('span.age').get_text(strip=True)
#     print(age)

    return {
        'title': title,
        'description': description,
        'author': author,
        'genre': [g.strip() for g in genre.split(',')],
        'age':age
    }


def episode_list(title_id):
    url = f'https://comic.naver.com/webtoon/list.nhn?titleId={title_id}'
#     print(url)
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html)
    
    results = []
    table = soup.select_one('table.viewList')
    tr_list = table.select('tr')
    
    for tr in tr_list:
        td_title = tr.select_one('td.title')
        
        if not td_title:
            continue
            
        title = td_title.get_text(strip=True)
        
        td_rating = tr.select_one('td:nth-child(3)')
        rating = td_rating.select_one('strong').get_text(strip=True)
        
        td_date = tr.select_one('td.num')
        date = td_date.get_text(strip=True)
        
        episode = {
            'title': title,
            'rating': rating,
            'date': date
        }
        results.append(episode)
    
    return results


def get_title_id(search_keyword):
    search_keyword = input('> 웹툰명을 입력하세요: ')
    results = webtoon_info(search_keyword)
#     print(results)

#     print('# 검색 결과')
    
    for index, result in enumerate(results, start=1):
        print(f'{index}: {result["title"]}')
        
    choice = int(input('> 웹툰 선택: '))
    
    webtoon = results[choice -1]
    title_id = webtoon['title_id']
    
    return title_id


while True:
    print(menu)
    val = input('> 입력:')
    
    if val == '1':
        title_id = get_title_id(val)
        info = webtoon_detail(title_id)
        
        print(f'# 정보 ({info["title"]})')
        print(f'설명: {info["description"]}')
        print(f'작가: {info["author"]}')
        print(f'장르: {info["genre"]}')
        print(f'연령제한: {info["age"]}')
        print()
        
    elif val == '2':
        title_id = get_title_id(val)
        episodes = episode_list(title_id)
        
        for episode in episodes:
            print(episode['title'])
            print(f'평점: {episode["rating"]}, 날짜: {episode["date"]}')
            print()
            
    elif val == '0':
        break

# 웹툰 크롤러 #
1. 정보
2. 에피소드 목록
0. 나가기
> 입력:1
> 웹툰명을 입력하세요: 삶
1: 삶이 우리를 속일지라도
> 웹툰 선택: 1
# 정보 (삶이 우리를 속일지라도)
설명: 재벌 3세들과 친해진 가난한 대학생!
그녀는 솔직해질 수 있을까?
작가: 김지수
장르: ['스토리', '드라마', '로맨스']
연령제한: 전체연령가

# 웹툰 크롤러 #
1. 정보
2. 에피소드 목록
0. 나가기
> 입력:2
> 웹툰명을 입력하세요: 구
1: 귀전구담
2: 이것도 친구라고
3: 내 여자친구는 상남자
4: 열정호구
5: 밤하늘에 구름운
6: 구독금지
7: 내 여자친구는 상남자
8: 옆집친구
9: 구원자
10: 대표님이 구독하셨습니다
11: 마루한 - 구현동화전
12: 구름이 피워낸 꽃
13: GO!구마 가족
> 웹툰 선택: 9
14화
평점: 6.96, 날짜: 2019.12.05

13화
평점: 7.23, 날짜: 2019.11.28

12화
평점: 8.02, 날짜: 2019.11.21

11화
평점: 8.59, 날짜: 2019.11.14

10화
평점: 8.19, 날짜: 2019.11.07

9화
평점: 6.95, 날짜: 2019.10.31

8화
평점: 7.46, 날짜: 2019.10.24

7화
평점: 7.66, 날짜: 2019.10.17

6화
평점: 6.42, 날짜: 2019.10.10

5화
평점: 6.70, 날짜: 2019.10.03

# 웹툰 크롤러 #
1. 정보
2. 에피소드 목록
0. 나가기
> 입력:0
